In [1]:
import numpy as np
import rerun as rr
import laspy
import os
import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_ndvi(red, nir):
    red = np.array(red, dtype=np.float64)
    nir = np.array(nir, dtype=np.float64)

    red = red / 65535
    nir = nir / 255

    return (nir - red) / (nir + 1e-8)

In [3]:
input_file = os.path.join('..', 'data', 'bws_100.LAZ')

with laspy.open(input_file) as f:
    las = f.read()

    red = las.red
    nir = las.nir


In [4]:
ndvi = get_ndvi(red, nir)

out_las = las
out_las.add_extra_dim(laspy.ExtraBytesParams(name='ndvi', type=np.float32))
out_las.ndvi = ndvi

In [12]:
# Filter 1: Filter out points with NDVI value under threshold
ndvi_threshold = -0.2
out_las = out_las[out_las.ndvi > ndvi_threshold]

# Filter 2: Remove points with single return
out_las = out_las[out_las.number_of_returns > 1]

# Filter 3: Remove last return of all remaining points
out_las = out_las[out_las.return_number != out_las.number_of_returns]


In [13]:
# Remove outliers using SOR
def remove_outliers(las_data, nb_neighbors=20, std_ratio=2.0):
    # Convert LAS data to Open3D point cloud
    xyz = np.vstack((las_data.x, las_data.y, las_data.z)).transpose()
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)

    # Perform statistical outlier removal
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    
    # Filter the original LAS data
    filtered_las = las_data[ind]
    
    return filtered_las

# After your previous filtering steps:
out_las = remove_outliers(out_las, 20, 2.0)

In [ ]:
def visualize_point_cloud_with_rerun(las_data):
    # Initialize rerun
    rr.init("LAS Point Cloud Viewer - NDVI, NIR, Red", spawn=True)

    # Check if red and nir bands exist
    if not (hasattr(las_data, 'red') and hasattr(las_data, 'nir')):
        raise ValueError("The LAS file does not contain 'red' and 'nir' bands required for NDVI calculation.")

    # Calculate NDVI
    ndvi = get_ndvi(las_data.red, las_data.nir)

    points = np.column_stack((las_data.x, las_data.y, las_data.z))
    ndvi_values = ndvi
    nir_values = las_data.nir
    red_values = las_data.red

    # Normalize NIR and Red values to 0-1 range
    nir_normalized = (nir_values - np.min(nir_values)) / (np.max(nir_values) - np.min(nir_values))
    red_normalized = (red_values - np.min(red_values)) / (np.max(red_values) - np.min(red_values))

    # Create color array: Red channel for Red values, Green for NDVI, Blue for NIR
    colors = np.column_stack((
        red_normalized,
        (ndvi_values + 1) / 2,  # NDVI is already in [-1, 1] range, normalize to [0, 1]
        nir_normalized
    ))

    # Log the point cloud to rerun
    rr.log("point_cloud", 
           rr.Points3D(
               positions=points,
               colors=colors,
               #radii=0.5,  # Adjust point size as needed
               labels=[f"NDVI: {ndvi:.3f}, NIR: {nir:.3f}, Red: {red:.3f}" 
                       for ndvi, nir, red in zip(ndvi_values, nir_values, red_values)]
           ))

    # Log some text to provide context
    rr.log("info", rr.TextLog("Point cloud: Red channel = Red band, Green channel = NDVI, Blue channel = NIR band. Hover for values."))

    print(f"Visualization ready. Run 'rerun' in terminal to view.")

Visualization ready. Run 'rerun' in terminal to view.


In [20]:
visualize_point_cloud_with_rerun(out_las)

Visualization ready. Run 'rerun' in terminal to view.
